## The Super Step
* A single iteration over the graph nodes. 
* Nodes that run in parallel are part of the same super-step, while nodes that run sequentially belong to separate super-steps. 
* The graph describles one super-step; one interaction between agents and tools to achieve an outcome. This means every user interactino is a fresh graph.invoke(state) call. 
* The reducer handles updating state during a super-step but not between super-steps. 

For examples
1. DEFINE GRAPH (the 5 step process of initalising a graph)
2. Super-step (user asks the chatbot about a question)
3. Super-step (user asks a follow up question)
4. ...

This is important as we need to keep checkpoints of each iteration (each iteration of a super-step) to diagonise how the model is thinking for each each iteration.

In [4]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.agents import Tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver

import sqlite3
from typing import Annotated, TypedDict
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr

import requests
import os

In [ ]:
load_dotenv(override=True)

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper
serper = GoogleSerperAPIWrapper()
serper.run("What is the capital of France?")

functions --> tool conversion (using langchain)
Tool has the following params
* name of the tool
* function which should be executed
* description of the function which is being executed.

In Lang graph the tools need to be used respectively
* Changes to provide the tools to OpenAI in json when we make the call
* Changes to handle the results back: look for the model staying that the finish_reason="tool_calls" and then retrieve the call, run the function, provide the results. 

In [ ]:
tool_search = Tool(
    name="search",
    func=serper.run,
    description="Useful for hwen you need more information form an online search."
)

# tool_search.invoke("What is the capital of France?")

In [ ]:
tools = [tool_search]

# Initalise LangGraph

initalise first state

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

Initalise graph with first state

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
# instead of creating a json for each tool, langchain packages by initalising it in another llm (easily retrieveable)
llm_with_tools = llm.bind_tools(tools)

create a node

In [ ]:
def chatbot(state: State):
    # packaging the tool into json version
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
# actual calling of the tool function (special node)
graph_builder.add_node("tools", ToolNode(tools=tools))

creating edges

In [ ]:
# conditional edge which calls tools based on an "if"
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")

# ontop of the creating the conditional edge, you need to still establish the edge connection with tools and chatbot
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

compile the graph

In [ ]:
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

Broken lines means, if the chatbot calls tools node, then execute the tools node and return back to the chatbot inital state of mind.

Run the following model in gradio

In [ ]:
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]})
    return result["messages"][-1].content

# gr.ChatInterface(chat, type="messages").launch()

Currently the model doesn't have retention for memory. This is due to a super step being considered as a single iteration over the graph nodes. Nodes that run in parallel are port of the same super-step, whilst nodes that run sequentially belong to separate super-steps.

In shorter terms 1 super step of the graph represents one invocation of passing message between the agent. You call invoke to run your graph for each super-step (for each interaction). The reducer handles state updates automatically within one super-step, but not between them. This is what checkpointing achieves (keeps with memory retention).

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Step 1
memory = MemorySaver()

In [ ]:
# 1 and 2
graph_builder = StateGraph(State)

# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}} # attachment to the first thread in memory.

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

The following is function which will show the history behind each super step. basically every time the graph is invoked (every interaction). 

In [ ]:
list(graph.get_state_history(config))

Furthermore, LangChain gives you tools to set the state back to a prior point in time, to branch off (based on a checkpoint id (from the history))

config = {"configurable": {"thread_id": "1", "checkpoint_id"}}

graph.invoke(None, config=config)

Furthermore, This would allow you to build a stable system that can be recovered (time travelled back similar to how you can edit a response in gpt) and re-run from a specified prior checkpoint in the config memory space. 

Also another to note, you can run this in parallel with another chatbot, by using a different memorysaver instances and creating repeating the steps again for the creating the langgraph structure.

EVEN BETTER YOU CAN CHANGE THE THREAD ID in the memory saver instance to talk another model with different instances of data retention. I Would assume that you can store these the retention values based on their thread id or thread number into an sql database (along with the chat history and super step history)

In [ ]:
# instead of memory saver we are using sqlite3 for saving memory
database_path = "/Users/goldenmeta/Github/uDemy/SQLite3/langgraph_sample.db"
connection = sqlite3.connect(database_path, check_same_thread=False)
sql_memory = SqliteSaver(connection)

In [ ]:
tool_search = Tool(
    name="search",
    func=serper.run,
    description="Useful for when you need more information from an online search."
)

tools = [tool_search]

In [ ]:
# Step 1 (Initial State)
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
# Step 2 (Create Inital Graph with Starting State)
graph_builder = StateGraph(State)

In [1]:
# Step 3 (Initialise the model and tools, and their respective nodes).
model = ChatOpenAI(model="gpt-4o-mini")
model_tools = model.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [model_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

NameError: name 'ChatOpenAI' is not defined